# Bronze to Silver
In this notebook the transformation from a bronze dataframe to silver dataframe will be performed
The tasks that will be executed are the following:
- Filter the years
- Drop countries with no GDP data
- Identify outliers and treat them
- Scale the data



In [76]:
import pandas as pd
import numpy as np
import os

read_path = os.getcwd() + '\Databases' #Path to your databases folder to be read
write_path = os.getcwd() + '\Output' #Path to the folder you want to store the dataframes

from Project.Utils.data_treat import iqr_treatment, nan_treatment

## Establish the variables and dataframe
Define variables that will be used along this notebook and import the Bronze dataframe generated in the previous notebook, which has been saved in the Output folder.

In [77]:
THRESHOLD = 20 #Minimum number of entries a country needs to have to be included into our research
year_min = 1990
year_max = 2020
columns_index = ('Country', 'Year')
column_year  = 'Year'
column_country = 'Country'
df = pd.read_csv(write_path + '/BronzeDataframe.csv')


C:\Users\smanoles\AppData\Local\Temp\ipykernel_3368\3551922364.py:7: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(write_path + '/BronzeDataframe.csv')


In [78]:
#Narrow the range of the data to the years selected
df[column_year]= df[column_year].astype(int)
df.drop(df[df[column_year] < year_min].index, inplace = True)
df.drop(df[df[column_year] > year_max].index, inplace = True)


In [79]:
print(df)

                       Country  Year  Gender Equality  % Undernourishment  \
6510               Afghanistan  1990              NaN                 NaN   
6511                   Albania  1990              NaN                 NaN   
6512                   Algeria  1990              NaN                 NaN   
6513            American Samoa  1990              NaN                 NaN   
6514                   Andorra  1990              NaN                 NaN   
...                        ...   ...              ...                 ...   
67198             Wuerttemburg  2015              NaN                 NaN   
67235      Yemen Arab Republic  2015              NaN                 NaN   
67236  Yemen People's Republic  2015              NaN                 NaN   
67237               Yugoslavia  2015              NaN                 NaN   
67523            Åland Islands  2015              NaN                 NaN   

       AgriShareGDP  CreditToAgriFishForest  EmploymentRural           GDP 

## Drop countries with no GDP data
Countries with really low information will be dropped from the research. A threshold of 20 entries must be satisfied in order to include the country. If the country doesnt have atleast 20 indicators will be deleted

In [80]:
df_countries = pd.read_csv(read_path + '/FAOSTAT_GDP.csv')
country_list = [] #List to insert all countries of the previous faostat csv

#Loop to insert all the possible different countries in to the list  
for country in df_countries['Area']:
    if country not in country_list:
        country_list.append(country)


for country in df['Country']:
    if country not in country_list:
        df.drop(df.loc[df['Country'] == country].index, inplace = True)

# Identifying outliers and treat
For each country and indicator we will identify if in first place has sufficient data. If the indicator has low data the iqr method won't be performed. If it has enough entries the iqr method will be called to treat the outliers.

In [81]:
for country in country_list:
        aux =  df.loc[df['Country'] == country].iloc[: , 2:]
        for column in aux.columns:
                if(aux[column].isna().sum() > 10):
                        aux[column] = np.nan
        #aux = nan_treatment(aux)

        print(aux)
        if(aux[column].isna().sum() < 10):
                aux = iqr_treatment(aux)

                
                    





       Gender Equality  % Undernourishment  AgriShareGDP  \
6510               NaN                 NaN           NaN   
6727               NaN                 NaN           NaN   
6944               NaN                 NaN           NaN   
7161               NaN                 NaN           NaN   
7378               NaN                 NaN           NaN   
7595               NaN                 NaN           NaN   
7812               NaN                 NaN           NaN   
8029               NaN                 NaN           NaN   
8246               NaN                 NaN           NaN   
8463               NaN                 NaN           NaN   
8680               NaN                 NaN           NaN   
8897               NaN                 NaN           NaN   
9114               NaN                 NaN           NaN   
9331               NaN                 NaN           NaN   
9548               NaN                 NaN           NaN   
9765               NaN                 N

IndexError: cannot do a non-empty take from an empty axes.